In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import datasets

In [2]:
data = datasets.load_breast_cancer()

In [3]:
X = data.data
Y = data.target

In [4]:
Y[Y==0] =-1
print(Y)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1  1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
  1  1  1  1  1 -1 -1  1 -1 -1  1  1  1  1 -1  1 -1 -1  1  1  1  1 -1  1
 -1 -1  1 -1  1 -1 -1  1  1  1 -1 -1  1 -1 -1 -1  1  1  1 -1  1  1 -1 -1
  1  1  1 -1 -1  1  1  1  1 -1  1  1 -1  1  1  1  1  1  1  1  1 -1 -1 -1
  1 -1 -1  1  1  1 -1 -1  1 -1  1 -1 -1  1 -1 -1  1  1 -1  1  1 -1  1  1
  1  1 -1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1 -1 -1  1 -1  1  1 -1
 -1  1  1 -1 -1  1  1  1  1 -1  1  1 -1 -1 -1  1 -1  1 -1  1  1  1 -1  1
  1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1  1 -1  1 -1  1  1 -1  1 -1 -1 -1 -1
  1  1 -1 -1  1  1  1 -1  1  1  1  1  1 -1 -1  1  1 -1  1  1 -1 -1  1 -1
  1  1  1  1 -1  1  1  1  1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1  1  1  1  1  1  1 -1  1 -1  1  1 -1  1  1 -1  1 -1 -1  1  1  1  1
  1  1  1  1  1  1  1  1  1 -1  1  1 -1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1 -1  1 -1  1  1  1  1 -1

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)
print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

(455, 30) (114, 30)
(455,) (114,)


In [7]:
Y_train = Y_train.reshape((-1,1))
Y_test = Y_test.reshape((-1,1))
print(Y_train.shape,Y_test.shape)

(455, 1) (114, 1)


In [22]:
class SVM:
    def __init__(self,C=1.0):
        self.C =C
        # Weights
        self.W =0
        # bias
        self.b =0
    # Loss -->W,T
    def hingeLoss(self,W,b,X,Y):
        loss =0.0
        loss+=0.5 *np.dot(W,W.T)
        # Iterate over all examples
        m = X.shape[0]
        for i in range(m):
            ti = Y[i] * (np.dot(W,X[i].T))
            loss+= self.C * max(0,(1-ti))
        return loss[0][0]
    def fit(self,X,Y,batch_size=100,lr=0.001,max_itr=300):
        no_of_features = X.shape[1]
        no_of_samples = X.shape[0]
        n = lr
        C= self.C
        
        # initializing parameter
        W = np.zeros((1,no_of_features))
        bias = 0
        print(self.hingeLoss(W,bias,X,Y))
        
        # Training
        losses = []
        for i in range(max_itr):
            l = self.hingeLoss(W,bias,X,Y)
            losses.append(l)
            
            ids = np.arange(no_of_samples)
            np.random.shuffle(ids)
            
            # Batch Gradient
            for batch_start in range(0,no_of_samples,batch_size):
                # 0 gradient for batch
                gradw =0
                gradb =0
                # Iterate over all examples in mini batch
                for j in range(batch_size,batch_start+batch_size):
                    if j<no_of_samples:
                        i = ids[j]
                        ti = Y[i] * (np.dot(W,X[i].T)+bias)
                        if ti<1:
                            gradw += C*Y[i]*X[i]
                            gradb += C*Y[i]
            W = W - n*W +n*gradw
            bias = bias + n*gradb
            
        self.W = W
        self.b = bias
            
        return W,bias,losses



In [9]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy
def predict(X,W,b):
        approx = np.dot(X,W.T) -b
        return np.sign(approx)

In [23]:
mySVM = SVM()
W,b,losses = mySVM.fit(X_train,Y_train,max_itr=300)
print(W)
print(b)
print(W.shape,X.shape)
y1= predict(X_train,W,b)
print(accuracy(y1,Y_train))
y2 = predict(X_test,W,b)
print(accuracy(y2,Y_test))

455.0
[[ 7.52594507e+01  1.33712843e+02  4.56860254e+02  4.38411837e+02
   7.89717985e-01  2.00277262e-01 -5.99710488e-01 -2.86978905e-01
   1.49369840e+00  5.93757483e-01  2.45060551e-01  1.02377240e+01
   4.97415344e-01 -1.78864857e+02  5.95096930e-02  7.29562627e-02
   6.21495415e-02  3.87345246e-02  1.63813185e-01  2.54782768e-02
   7.46289296e+01  1.74206747e+02  4.49666619e+02 -4.68842207e+02
   1.04480876e+00  1.66012222e-01 -8.32893557e-01 -1.38175586e-01
   2.16290970e+00  6.53906932e-01]]
[11.597]
(1, 30) (569, 30)
0.8967032967032967
0.8596491228070176


In [26]:
mySVM = SVM(C=0.5)
W,b,losses = mySVM.fit(X_train,Y_train,max_itr=300)
y1= predict(X_train,W,b)
print(accuracy(y1,Y_train))
y2 = predict(X_test,W,b)
print(accuracy(y2,Y_test))

227.5
0.8549450549450549
0.8508771929824561


In [27]:
mySVM = SVM(C=0.8)
W,b,losses = mySVM.fit(X_train,Y_train,max_itr=300)
y1= predict(X_train,W,b)
print(accuracy(y1,Y_train))
y2 = predict(X_test,W,b)
print(accuracy(y2,Y_test))

364.000000000003
0.9098901098901099
0.8859649122807017


In [20]:
mySVM = SVM()
W,b,losses = mySVM.fit(X_train,Y_train,max_itr=300)
print(W)
print(b)
print(W.shape,X.shape)
y1= predict(X_train,W,b)
print(accuracy(y1,Y_train))
y2 = predict(X_test,W,b)
print(accuracy(y2,Y_test))

455.0
[[ 7.66481743e+01  1.35925072e+02  4.65375767e+02  4.48128333e+02
   8.00995063e-01  2.04669048e-01 -6.07456509e-01 -2.89989485e-01
   1.51764075e+00  6.02856177e-01  2.03311424e-01  1.03370290e+01
   1.92325540e-01 -1.90006427e+02  5.99469622e-02  7.38423414e-02
   6.21846878e-02  3.93514500e-02  1.66020227e-01  2.58230925e-02
   7.59560856e+01  1.77225816e+02  4.58047380e+02 -4.89188468e+02
   1.06287335e+00  1.94562398e-01 -8.23284965e-01 -1.31194542e-01
   2.21406061e+00  6.66808262e-01]]
[11.738]
(1, 30) (569, 30)
0.8571428571428571
0.8508771929824561
